In [24]:
import gempyor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import copy

# import seaborn as sns
import matplotlib._color_data as mcd
import pyarrow.parquet as pq
import click
import subprocess
import dask.dataframe as dd
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from matplotlib.backends.backend_pdf import PdfPages
from gempyor.utils import config, as_list
import os
import multiprocessing as mp
import pandas as pd
import pyarrow.parquet as pq
import xarray as xr
from gempyor import config, model_info, outcomes, seir, inference_parameter, logloss, inference
from gempyor.inference import GempyorInference
import gempyor.postprocess_inference
import multiprocessing, shutil

import os
from multiprocessing import cpu_count

# disable  operations using the MKL linear algebra.
os.environ["OMP_NUM_THREADS"] = "1"
import emcee

from multiprocessing import Pool

## Test A
project_path = "./flepiMoP/examples/simple_usa_statelevel/"  # necessary for now for time serie parameters !!! FIXME
config_filepath = f"simple_usa_statelevel.yml"
seir_modifiers_scenario="Ro_all"
outcome_modifiers_scenario=None

# Test B
#project_path = "RSV_USA/"
#config_filepath = f"config_rsvnet_emcee.yml"
#seir_modifiers_scenario="none"
#outcome_modifiers_scenario="IHRadj"

# Test C
project_path = "COVID19_USA/"
config_filepath= "config_SMH_R18_allBoo_highIE_blk1_emcee.yml"
#config_filepath="config_FCH_R17_allBoo_highIE_phase5_blk1_21Apr2024.yml"


nwalkers = 3
niter = 4
nsamples = 50 # number of likelyhood eval to write to disk...
nthin=5
run_id = "calib_R18"

#### START COPY PAST FROM flepiMoP/flepimop/gempyor_pkg/src/gempyor/calibrate.py
ncpu = cpu_count()
print(f"found {ncpu} CPUs, using these")

gempyor_inference = GempyorInference(
        config_filepath=config_filepath,
        run_id=run_id,
        prefix=None,
        first_sim_index=1,
        stoch_traj_flag=False,
        rng_seed=None,
        nslots=1,
        inference_filename_prefix="",  # usually for {global or chimeric}/{intermediate or final}
        inference_filepath_suffix="",  # usually for the slot_id
        out_run_id=None,  # if out_run_id is different from in_run_id, fill this
        out_prefix=None,  # if out_prefix is different from in_prefix, fill this
        path_prefix=project_path,  # in case the data folder is on another directory
        autowrite_seir=False,
)

#inferpar = inference_parameter.InferenceParameters(global_config=config, modinf=modinf)
p0 = gempyor_inference.inferpar.draw_initial(n_draw=nwalkers)
for i in range(nwalkers):
    assert  gempyor_inference.inferpar.check_in_bound(proposal=p0[i]), "The initial parameter draw is not within the bounds, check the perturbation distributions"

nsubpop = len(gempyor_inference.modinf.subpop_struct.subpop_names)
subpop_names = gempyor_inference.modinf.subpop_struct.subpop_names


print(gempyor_inference.logloss)
print(gempyor_inference.inferpar)

found 12 CPUs, using these
  gempyor >> Running ***DETERMINISTIC*** simulation;
  gempyor >> ModelInfo USA_inference_all; index: 1; run_id: calib_R18,
  gempyor >> prefix: USA/calib_R18/;
Loaded subpops in loaded relative probablity file: 51 Intersect with seir simulation:  51 kept
Running Gempyor Inference
LogLoss: 2 statistics and 10710 data points,number of NA for each statistic: 
source        0
incidC    10710
cumC      10710
incidD        0
cumD          0
incidH       51
cumH         51
dtype: int64
InferenceParameters: with 714 parameters: 
    seir_modifiers: 612 parameters
    outcome_modifiers: 102 parameters

LogLoss: 2 statistics and 10710 data points,number of NA for each statistic: 
source        0
incidC    10710
cumC      10710
incidD        0
cumD          0
incidH       51
cumH         51
dtype: int64
InferenceParameters: with 714 parameters: 
    seir_modifiers: 612 parameters
    outcome_modifiers: 102 parameters



In [46]:
snpi_df_mod, hnpi_df_mod = gempyor_inference.inferpar.inject_proposal(proposal=p0[0], 
                                                                        snpi_df=gempyor_inference.static_sim_arguments["snpi_df_ref"], 
                                                                        hnpi_df = gempyor_inference.static_sim_arguments["hnpi_df_ref"])
    
ss = copy.deepcopy(gempyor_inference.static_sim_arguments)
ss["snpi_df_in"] = snpi_df_mod
ss["hnpi_df_in"] = hnpi_df_mod
del ss["snpi_df_ref"]
del ss["hnpi_df_ref"]

    
outcomes_df = gempyor.inference.simulation_atomic(**ss, modinf=gempyor_inference.modinf, save=False)


In [82]:
outcomes_df.reset_index()

,subpop,incidI_unvaccinated_ALL_age0to17ph4,incidI_vaccinated_ALL_age0to17ph4,incidI_unvaccinated_ALL_age18to64HRph4,incidI_vaccinated_ALL_age18to64HRph4,incidI_unvaccinated_ALL_age18to64LRph4,incidI_vaccinated_ALL_age18to64LRph4,incidI_unvaccinated_ALL_age65to100ph4,incidI_vaccinated_ALL_age65to100ph4,incidI_ALL,...,incidD_unvaccinated_ALL_age0to17ph4,incidD_vaccinated_ALL_age0to17ph4,incidD_unvaccinated_ALL_age18to64HRph4,incidD_vaccinated_ALL_age18to64HRph4,incidD_unvaccinated_ALL_age18to64LRph4,incidD_vaccinated_ALL_age18to64LRph4,incidD_unvaccinated_ALL_age65to100ph4,incidD_vaccinated_ALL_age65to100ph4,incidD_ALL,incidD
date,,,,,,,,,,,,,,,,,,,,,
2023-03-19,56000,161.155649,4.860670,76.573181,10.004146,293.345569,38.325063,63.291660,47.102563,694.658502,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-20,56000,161.155649,4.860670,76.573181,10.004146,293.345569,38.325063,63.291660,47.102563,694.658502,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-21,56000,163.445375,4.954448,77.655026,10.188774,297.490029,39.032358,64.175853,47.943667,704.885530,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-22,56000,163.445375,4.954448,77.655026,10.188774,297.490029,39.032358,64.175853,47.943667,704.885530,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-23,56000,190.023665,5.789776,90.278189,11.898338,345.848329,45.581555,74.601693,55.960694,819.982239,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-27,06000,3486.289508,1299.878629,1410.270870,1296.461716,6148.809297,4478.114864,777.593552,2364.990430,21262.408866,...,0.074010,0.011802,8.199077,3.230311,3.844055,1.199822,35.990992,46.913089,99.463158,298.389473
2025-08-28,06000,3486.289508,1299.878629,1410.270870,1296.461716,6148.809297,4478.114864,777.593552,2364.990430,21262.408866,...,0.074010,0.011802,8.199077,3.230311,3.844055,1.199822,35.990992,46.913089,99.463158,298.389473
2025-08-29,06000,3350.064223,1249.086387,1355.165135,1245.802996,5908.547182,4303.134348,747.209415,2272.579410,20431.589096,...,0.065270,0.010408,7.230826,2.848835,3.390100,1.058132,31.740723,41.373000,87.717295,263.151886


In [78]:
# find the
reg = .9
mult=2
print("reg is", reg)
for sp in outcomes_df["subpop"].unique():
    max_fit = outcomes_df[outcomes_df["subpop"]==sp]["incidC"][:"2024-04-08"].max()*reg
    max_summer = outcomes_df[outcomes_df["subpop"]==sp]["incidC"]["2024-04-08":"2024-09-30"].max()  
    if max_summer > max_fit:
        print(f"changing {sp} because max_summer max_summer={max_summer:.1f} > reg*max_fit={max_fit:.1f}, diff {max_fit/max_summer*100:.1f}%")
        print(f">>> MULT BY {max_summer/max_fit*mult:2f}")
        outcomes_df.loc[outcomes_df["subpop"]==sp, ["incidH", "incidD"]] = outcomes_df.loc[outcomes_df["subpop"]==sp, ["incidH", "incidD"]]*max_summer/max_fit*mult

reg is 0.9
changing 50000 because max_summer max_summer=10764.7 > reg*max_fit=9693.5, diff 90.0%
>>> MULT BY 2.221024
changing 46000 because max_summer max_summer=1113.2 > reg*max_fit=978.5, diff 87.9%
>>> MULT BY 2.275169
changing 10000 because max_summer max_summer=25576.2 > reg*max_fit=24704.6, diff 96.6%
>>> MULT BY 2.070561
changing 30000 because max_summer max_summer=4801.2 > reg*max_fit=4400.2, diff 91.6%
>>> MULT BY 2.182259
changing 33000 because max_summer max_summer=14530.6 > reg*max_fit=12363.2, diff 85.1%
>>> MULT BY 2.350614
changing 05000 because max_summer max_summer=59917.8 > reg*max_fit=57489.6, diff 95.9%
>>> MULT BY 2.084474
changing 09000 because max_summer max_summer=41027.8 > reg*max_fit=35489.7, diff 86.5%
>>> MULT BY 2.312099
changing 40000 because max_summer max_summer=6761.0 > reg*max_fit=4239.5, diff 62.7%
>>> MULT BY 3.189478
changing 41000 because max_summer max_summer=42154.6 > reg*max_fit=40232.5, diff 95.4%
>>> MULT BY 2.095552
changing 24000 because ma

1.4459537302466698

In [60]:
outcomes_df.loc[outcomes_df["subpop"]==sp, ["incidD", "incidH"]]

,incidD,incidH
date,,
2023-03-19,0.000000,0.000000
2023-03-20,0.000000,0.000000
2023-03-21,0.000000,0.000000
2023-03-22,0.000000,0.000000
2023-03-23,0.000000,0.000000
...,...,...
2025-08-27,298.389473,669.908975
2025-08-28,298.389473,625.093566
2025-08-29,263.151886,625.093566


In [56]:
outcomes_df.loc?

Type:        property
String form: <property object at 0x127f72480>
Docstring:  
Access a group of rows and columns by label(s) or a boolean array.

``.loc[]`` is primarily label based, but may also be used with a
boolean array.

Allowed inputs are:

- A single label, e.g. ``5`` or ``'a'``, (note that ``5`` is
  interpreted as a *label* of the index, and **never** as an
  integer position along the index).
- A list or array of labels, e.g. ``['a', 'b', 'c']``.
- A slice object with labels, e.g. ``'a':'f'``.

  .. warning:: Note that contrary to usual python slices, **both** the
      start and the stop are included

- A boolean array of the same length as the axis being sliced,
  e.g. ``[True, False, True]``.
- An alignable boolean Series. The index of the key will be aligned before
  masking.
- An alignable Index. The Index of the returned selection will be the input.
- A ``callable`` function with one argument (the calling Series or
  DataFrame) and that returns valid output for inde

In [ ]:
outcomes_df[outcomes_df["subpop"]==sp]

,subpop,incidI_unvaccinated_ALL_age0to17ph4,incidI_vaccinated_ALL_age0to17ph4,incidI_unvaccinated_ALL_age18to64HRph4,incidI_vaccinated_ALL_age18to64HRph4,incidI_unvaccinated_ALL_age18to64LRph4,incidI_vaccinated_ALL_age18to64LRph4,incidI_unvaccinated_ALL_age65to100ph4,incidI_vaccinated_ALL_age65to100ph4,incidI_ALL,...,incidD_unvaccinated_ALL_age0to17ph4,incidD_vaccinated_ALL_age0to17ph4,incidD_unvaccinated_ALL_age18to64HRph4,incidD_vaccinated_ALL_age18to64HRph4,incidD_unvaccinated_ALL_age18to64LRph4,incidD_vaccinated_ALL_age18to64LRph4,incidD_unvaccinated_ALL_age65to100ph4,incidD_vaccinated_ALL_age65to100ph4,incidD_ALL,incidD
date,,,,,,,,,,,,,,,,,,,,,
2023-03-19,06000,9646.483398,1092.312891,4257.773795,1622.792388,16716.481352,6371.258785,2825.192504,3733.395954,46265.691067,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-20,06000,9646.483398,1092.312891,4257.773795,1622.792388,16716.481352,6371.258785,2825.192504,3733.395954,46265.691067,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-21,06000,22648.905955,2597.526704,9992.731664,3849.549472,39232.547470,15113.748419,6626.412560,8840.783038,108902.205284,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-22,06000,22648.905955,2597.526704,9992.731664,3849.549472,39232.547470,15113.748419,6626.412560,8840.783038,108902.205284,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-23,06000,40158.970524,4630.964345,17718.008124,6858.827243,69562.820074,26928.499078,11748.889556,15743.945429,193350.924373,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-27,06000,3486.289508,1299.878629,1410.270870,1296.461716,6148.809297,4478.114864,777.593552,2364.990430,21262.408866,...,0.074010,0.011802,8.199077,3.230311,3.844055,1.199822,35.990992,46.913089,99.463158,99.463158
2025-08-28,06000,3486.289508,1299.878629,1410.270870,1296.461716,6148.809297,4478.114864,777.593552,2364.990430,21262.408866,...,0.074010,0.011802,8.199077,3.230311,3.844055,1.199822,35.990992,46.913089,99.463158,99.463158
2025-08-29,06000,3350.064223,1249.086387,1355.165135,1245.802996,5908.547182,4303.134348,747.209415,2272.579410,20431.589096,...,0.065270,0.010408,7.230826,2.848835,3.390100,1.058132,31.740723,41.373000,87.717295,87.717295


In [ ]:
outcomes_df[outcomes_df["subpop"]==sp

In [19]:
from mpi4py.futures import MPIPoolExecutor
MPIPoolExecutor?

Init signature: MPIPoolExecutor(max_workers=None, initializer=None, initargs=(), **kwargs)
Docstring:      MPI-based asynchronous executor.
Init docstring:
Initialize a new MPIPoolExecutor instance.

Args:
    max_workers: The maximum number of MPI processes that can be used
        to execute the given calls. If ``None`` or not given then the
        number of worker processes will be determined from the MPI
        universe size attribute if defined, otherwise a single worker
        process will be spawned.
    initializer: An callable used to initialize workers processes.
    initargs: A tuple of arguments to pass to the initializer.

Keyword Args:
    python_exe: Path to Python executable used to spawn workers.
    python_args: Command line arguments to pass to Python executable.
    mpi_info: Dict or iterable with ``(key, value)`` pairs.
    globals: Dict or iterable with global variables to set in workers.
    main: If ``False``, do not import ``__main__`` in workers.
    path: 

In [5]:
filename = f"{run_id}_backend.h5"
sampler = emcee.backends.HDFBackend(filename, read_only=True)

In [21]:
a = [1,2]
a

[1, 2]

In [23]:
b,c = [1,2]
b

1

In [6]:


shutil.rmtree("model_output/", ignore_errors=True)
shutil.rmtree(project_path + "model_output/", ignore_errors=True)

max_indices = np.argsort(sampler.get_log_prob()[-1, :])[-nsamples:]
chains = sampler.get_chain()[:, max_indices, :]  # the last iteration, for selected slots
llik = sampler.get_log_prob()[:, max_indices]

gempyor.postprocess_inference.plot_chains(
        inferpar=gempyor_inference.inferpar, chains=chains, llik=llik, sampled_slots=None, save_to=f"{run_id}_chains.pdf"
    )
print("EMCEE Run done, doing sampling")

generating chain plot


100%|██████████| 51/51 [00:37<00:00,  1.35it/s]

EMCEE Run done, doing sampling


In [7]:
shutil.rmtree("model_output/", ignore_errors=True)
shutil.rmtree(project_path + "model_output/", ignore_errors=True)

samples = sampler.get_chain()[-1, max_indices, :]  # the last iteration, for selected slots
gempyor_inference.set_save(True)
with multiprocessing.Pool(ncpu) as pool:
    results = pool.starmap(
        gempyor_inference.get_logloss_as_single_number, [(samples[i, :],) for i in range(len(max_indices))]
    )

In [16]:
results = []
for fn in gempyor.utils.list_filenames(folder=project_path+"model_output/", filters=[run_id, "hosp.parquet"]):
   df = gempyor.read_df(fn)
   df = df.set_index("date")
   results.append(df)

In [18]:
def plot_fit(modinf, inferpar, loss, list_of_df, save_to):
    with PdfPages(f"{save_to}") as pdf:
            d = pdf.infodict()
            d["Title"] = "FlepiMoP Inference Fit"
            d["Author"] = "FlepiMoP Inference"
            

            for j, subpop in enumerate(modinf.subpop_struct.subpop_names):
                fig, axes = plt.subplots(1, len(loss.statistics), figsize=(3 * len(loss.statistics), 4), sharex=True)
                
                gt_s = loss.gt[loss.gt["subpop"] == subpop].sort_index()
                first_date = max(gt_s.index.min(), list_of_df[0].index.min())
                last_date = min(gt_s.index.max(), list_of_df[0].index.max())
                gt_s = gt_s.loc[first_date:last_date].drop(["subpop"], axis=1).resample("W-SAT").sum()

                for i, (stat_name, stat) in enumerate(loss.statistics.items()):
                    ax = axes[i]
                    ax.plot(gt_s[stat.data_var], color="k", marker=".", lw=1)
                    for model_df in list_of_df:
                        model_df_s = (
                            model_df[model_df["subpop"] == subpop]
                            .drop(["subpop"], axis=1)
                            .loc[first_date:last_date]
                            .resample("W-SAT")
                            .sum()
                        )  # todo sub subpop here
                        ax.plot(model_df_s[stat.sim_var], lw=0.9, alpha=0.5)
                    # if True:
                    #        init_df_s = outcomes_df_ref[model_df["subpop"]==subpop].drop(["subpop","time"],axis=1).loc[min(gt_s.index):max(gt_s.index)].resample("W-SAT").sum() # todo sub subpop here
                    ax.set_title(f"{stat_name}, {subpop}")
                fig.tight_layout()
                pdf.savefig(fig)
                plt.close(fig)
plot_fit(modinf=gempyor_inference.modinf, inferpar=gempyor_inference.inferpar, loss=gempyor_inference.logloss, list_of_df=results, save_to=f"{run_id}_fit.pdf")

In [29]:
samples = sampler.get_chain()[-1,max_indices ,:]
a = [(samples[i,:],) for i in range(len(max_indices))]

In [ ]:
simulation_atomic(**ss, modinf=self.modinf, save=self.save)

In [3]:
gempyor_inference.perform_test_run()

test run successful 🎉, with logloss=-45965.6 including -5500.4 for regularization (12.0%) 


(           subpop  incidI_unvaccinated_ALL_age0to17ph4  \
 date                                                     
 2023-03-19  56000                           110.394339   
 2023-03-20  56000                           110.394339   
 2023-03-21  56000                           132.331477   
 2023-03-22  56000                           132.331477   
 2023-03-23  56000                           142.058054   
 ...           ...                                  ...   
 2025-08-27  06000                         16035.451907   
 2025-08-28  06000                         16035.451907   
 2025-08-29  06000                         16634.840639   
 2025-08-30  06000                         16634.840639   
 2025-08-31  06000                         17256.442292   
 
             incidI_vaccinated_ALL_age0to17ph4  \
 date                                            
 2023-03-19                           4.605727   
 2023-03-20                           4.605727   
 2023-03-21                    

In [4]:
filename = f"{run_id}_backend.h5"
backend = emcee.backends.HDFBackend(filename)
backend.reset(nwalkers, gempyor_inference.inferpar.get_dim())
gempyor_inference.set_silent(False)
with Pool(3) as pool:
        sampler = emcee.EnsembleSampler(nwalkers, 
                                        gempyor_inference.inferpar.get_dim(), 
                                        gempyor_inference.get_logloss_as_single_number, 
                                        pool=pool,
                                        backend=backend, moves=[(emcee.moves.StretchMove(live_dangerously=True), 1)])
        state = sampler.run_mcmc(p0, niter, progress=True, skip_initial_state_check=True)

llik is -45729.70247489614
llik is -45842.41667236088
llik is -45976.52186496037


  0%|          | 0/4 [00:00<?, ?it/s]

llik is -45757.469693276245
llik is -45865.942736300945


 25%|██▌       | 1/4 [00:32<01:38, 32.96s/it]

llik is -45972.74954846854
llik is -45591.36506356454
llik is -45901.314939441305


 50%|█████     | 2/4 [01:05<01:05, 32.74s/it]

llik is -46074.59348286781
llik is -45890.32092923346
llik is -45969.61363397053


 75%|███████▌  | 3/4 [01:38<00:33, 33.00s/it]

llik is -45519.50216712484
llik is -45544.68410035816
llik is -45883.33418188836


In [ ]:
filename = f"{run_id}_backend.h5"
backend = emcee.backends.HDFBackend(filename)
backend.reset(3, inferpar.get_dim())
with Pool(3) as pool:
        sampler = emcee.EnsembleSampler(3, 
                                        inferpar.get_dim(), 
                                        a.run, 
                                        pool=pool,
                                        backend=backend, moves=[(emcee.moves.StretchMove(live_dangerously=True), 1)])
        state = sampler.run_mcmc(p0[0:3,:], 5, progress=True, skip_initial_state_check=True)

llik: -46237.56540759982
llik: -46051.281202656246
llik: -45831.8716673561


  0%|          | 0/5 [00:00<?, ?it/s]

llik: -46099.484509051355
llik: -46195.55396420405


 20%|██        | 1/5 [00:33<02:13, 33.33s/it]

llik: -45909.410404363836


 20%|██        | 1/5 [00:34<02:17, 34.33s/it]


KeyboardInterrupt: 

In [ ]:
ds["incidH_18_49"].sum(dim="subpop", skipna=False).plot()

In [ ]:
ds["incidH_18_49"].resample(date="W-SAT").sum().plot.line(x="date");

In [ ]:
ds["incidH_18_49"].resample(date="W-SAT").sum(skipna=False).plot.line(x="date");

In [ ]:
ds["incidH_18_49"].resample(date="W-SAT", skipna=False).sum().plot.line(x="date");

In [ ]:
if False:
    log_loss.plot_gt(subplot=True);
    for reg_func, reg_config in log_loss.statistics['incidCase'].regularizations:
        print(reg_func,"\n", reg_config)

## Create the first gempyor object

In [ ]:
hosp.reset_index().set_index(["date","subpop"])
xr.Dataset.from_dataframe(hosp.reset_index().set_index(["date","subpop"]))

In [ ]:
raise ValueError

## Run EMCEE

In [ ]:
backend = emcee.backends.HDFBackend(filename)
with Pool(ncpu) as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, 
                                    args=[snpi_df_ref, ndim, statistics, fitted_params, gt, hnpi_df_ref, modinf, p_draw, unique_strings, transition_array, proportion_array, proportion_info, initial_conditions, seeding_data, seeding_amounts,outcomes_parameters, True], 
                                    pool=pool,
                                    backend=backend,  moves=[(emcee.moves.StretchMove(live_dangerously=True), 1)])
    state = sampler.run_mcmc(None, (nsamples//nwalkers)+1, progress=True, skip_initial_state_check=True)

In [ ]:
    print(f"Done, mean acceptance fraction: {np.mean(sampler.acceptance_fraction):.3f}")

In [ ]:
plt.plot(sampler.acceptance_fraction)

## Analysi

In [ ]:
filename = f"{run_id}_backend.h5"
sampler = emcee.backends.HDFBackend(filename, read_only=True)

In [ ]:
sampler.get_chain().shape

In [ ]:
# todo: restart form other shape...

In [ ]:
last_llik = sampler.get_log_prob()[-1,:]
good_slots = last_llik > (last_llik.mean()-1*last_llik.std())
print(f"there are {good_slots.sum()}/{len(good_slots)} good walkers... keeping these")

In [ ]:
fig, axes = plt.subplots(inferpar.get_dim()+1,2, figsize=(15, (inferpar.get_dim()+1)*2))

labels = list(zip(inferpar.pnames, inferpar.subpops))
samples = sampler.get_chain()

import seaborn as sns
def plot_chain(frompt,axes):
    ax = axes[0]

    ax.plot(np.arange(frompt,frompt+sampler.get_log_prob()[frompt:].shape[0]),
                      sampler.get_log_prob()[frompt:,good_slots], "navy", alpha=.2, lw=1, label="good walkers")
    ax.plot(np.arange(frompt,frompt+sampler.get_log_prob()[frompt:].shape[0]),
            sampler.get_log_prob()[frompt:,~good_slots], "tomato", alpha=.4, lw=1, label="bad walkers")
    ax.set_title("llik")
    #ax.legend()
    sns.despine(ax=ax, trim=False)
    ax.set_xlim(frompt, frompt+sampler.get_log_prob()[frompt:].shape[0])

    #ax.set_xlim(0, len(samples))

    for i in range(inferpar.get_dim()):
        ax = axes[i+1]
        ax.plot(np.arange(frompt,frompt+sampler.get_log_prob()[frompt:].shape[0]),
                samples[frompt:,good_slots, i], "navy", alpha=.2, lw=1,)
        ax.plot(np.arange(frompt,frompt+sampler.get_log_prob()[frompt:].shape[0]),
                samples[frompt:, ~good_slots, i], "tomato", alpha=.4, lw=1,)
        #ax.set_xlim(0, len(samples))
        ax.set_title(labels[i])
        #ax.yaxis.set_label_coords(-0.1, 0.5)
        sns.despine(ax=ax, trim=False)
        ax.set_xlim(frompt, frompt+samples[frompt:].shape[0])
        

    axes[-1].set_xlabel("step number");

plot_chain(0,axes[:,0])
plot_chain(3*samples.shape[0]//4,axes[:,1])
fig.tight_layout()

plt.savefig(f"{run_id}_chains.pdf")

In [ ]:
good_samples =  sampler.get_chain()[:,good_slots,:]
good_samples.shape

In [ ]:
step_number = -1
exported_samples = np.empty((nsamples,ndim))
for i in range(nsamples):
    exported_samples[i,:] = good_samples[step_number - nthin*(i//(good_slots.sum())) ,i%(good_slots.sum()),:] # parentesis around i//(good_slots.sum() are very important

In [ ]:
position_arguments = [snpi_df_ref, ndim, statistics, fitted_params, gt, hnpi_df_ref, modinf, p_draw, unique_strings, transition_array, proportion_array, proportion_info, initial_conditions, seeding_data, seeding_amounts,outcomes_parameters, True]

with Pool(ncpu) as pool:
    results = pool.starmap(log_prob, [(sample, *position_arguments) for sample in exported_samples])

In [ ]:
np.mean(sampler.get_log_prob()[-1,:])

In [ ]:
#import concurrent.futures
#
def evaluate(proposals, k):
    proposal = proposals[k]
    snpi_df_mod, hnpi_df_mod = input_proposal(proposal, snpi_df_ref, hnpi_df_ref, fitted_params, ndim)

    outcomes_df = run_simulation(snpi_df_mod, 
                                hnpi_df_mod,
                                modinf, p_draw, unique_strings, transition_array, proportion_array, proportion_info, initial_conditions, seeding_data, seeding_amounts,outcomes_parameters)
    return outcomes_df
#
#with concurrent.futures.ThreadPoolExecutor(max_workers=nwalkers) as executor:
#    # Map the function to the list of k values in parallel
#    results = list(executor.map(lambda k_val: evaluate(samples[-1,:,:], k_val), np.arange(nwalkers)))
#

results = []
for k in range(nwalkers):
    results.append(evaluate(samples[-1,:,:], k))

In [ ]:
results

In [ ]:
results = []
for fn in gempyor.utils.list_filenames(folder="model_output/", filters=[run_id,"hosp.parquet"]):
    if "000000000" not in fn:
        df = gempyor.read_df(fn)
        #raise ValueError
        df["date"] = df["time"]
        df = df.set_index("date")
        results.append(df)

In [ ]:
results[0].index.max()

In [ ]:
fig, axes = plt.subplots(len(statistics),len(subpop_names), figsize=(5*len(subpop_names), 3*len(statistics)), sharex=True, dpi=300)
for j, subpop in enumerate(modinf.subpop_struct.subpop_names):
        gt_s = gt[gt["subpop"]==subpop].sort_index()
        first_date = max(gt_s.index.min(),results[0].index.min())
        last_date = min(gt_s.index.max(), results[0].index.max())
        gt_s = gt_s.loc[first_date:last_date].drop(["subpop"],axis=1).resample("W-SAT").sum()
        
        for i, (key, value) in enumerate(statistics.items()):
                ax = axes[i,j]
                ax.plot(gt_s[value], color='k', marker='.', lw=1)
                for model_df in results:
                        model_df_s = model_df[model_df["subpop"]==subpop].drop(["subpop","time"],axis=1).loc[first_date:last_date].resample("W-SAT").sum() # todo sub subpop here
                        ax.plot(model_df_s[key],  lw=.9, alpha=.5)
                #if True:
                #        init_df_s = outcomes_df_ref[model_df["subpop"]==subpop].drop(["subpop","time"],axis=1).loc[min(gt_s.index):max(gt_s.index)].resample("W-SAT").sum() # todo sub subpop here
                ax.set_title(f"{value}, {subpop}")
fig.tight_layout()
plt.savefig(f"{run_id}_results.pdf")

In [ ]:
gt_s.isna().sum()

In [ ]:
gt_s.plot(subplots=True)

In [ ]:
gt_s["incidH_65_100"]

In [ ]:
gt_s

In [ ]:
emcee.__version__

In [ ]:
import corner

In [ ]:
#tau = sampler.get_autocorr_time()
#print(tau)

flat_samples = sampler.get_chain(discard=30, thin=2,flat=True) #discard=100, thin=15, 
print(flat_samples.shape)

import corner

fig = corner.corner(
    flat_samples, labels=labels, #truths=[m_true, b_true, np.log(f_true)]
);